In [1]:
from pyagnps import aims
from pathlib import Path

Please install GDAL (see install_gdal.py). The subannagnps module will not work without it
Please install GDAL (see install_gdal.py). The topagnps module will not work without it


In [2]:
# Database credentials
credentials = Path("../../inputs/db_credentials.json")

In [3]:
output_folder = Path("C:/Users/Luc/Desktop/test_oow")

In [4]:
watershed0 = aims.AIMSWatershed(
                credentials,
                watershed_name="Goodwin Creek",
                watershed_description="As generated by AIMS",
                watershed_location="Mississippi",
                outlet_x=-89.91651,
                outlet_y=34.23151,
                start_date="2000-01-01",
                end_date="2000-12-31",
                # selected_reaches_for_output=["OUTLET"],
                climate_method="nldas2_database",
                output_folder=output_folder,
                export_gis=True)

In [5]:
path_to_cmip5_data_dir = Path("D:/AIMS/Datasets/Climate/CMIP/CMIP5/MACAv2METDATA/CNRM-CM5/r1i1p1")
path_to_cmip5_station_points_clim_id = path_to_cmip5_data_dir / "cmip5_maca_v2_metdata_pts_clim_ids.gpkg"

In [6]:
watershed0.get_thuc_id_by_xy()
watershed0.set_reaches_for_output(None)

In [7]:
watershed0.load_static_files()

c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: File /vsimem/eefe55dcdc054f6dab7238a3c022e575 has GPKG application_id, but non conformant file extension
  return ogr_read(
c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: File /vsimem/145d618cbe214881b8732c950d6b7c08 has GPKG application_id, but non conformant file extension
  return ogr_read(
c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: File /vsimem/7f507a1a093d40e09b0e18bf0243e0ae has GPKG application_id, but non conformant file extension
  return ogr_read(


In [8]:
input_folders = watershed0.make_watershed_input_dirs()

In [9]:
watershed0.query_cells()


In [10]:
watershed0.query_reaches()

In [11]:
watershed0.query_soil()

In [12]:
watershed0.query_management_field()
watershed0.query_management_schedule()
watershed0.query_management_crop()
watershed0.query_management_crop_growth()
watershed0.query_management_non_crop()
watershed0.query_management_operation()
watershed0.query_runoff_curve()

In [13]:
# watershed0.generate_climate_daily_files(date_mode="local",
#                                         overwrite=True,
#                                         path_to_cmip_dir=path_to_cmip5_data_dir,
#                                         path_to_cmip_station_points_id=path_to_cmip5_station_points_clim_id)

watershed0.generate_climate_daily_files(date_mode="local",
                                        overwrite=True,
                                        table="climate_nldas2")

c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
  0%|          | 0/4 [01:00<?, ?it/s]


ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: text = integer
LINE 17:                 station_id = 34617
                                    ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: 
            SELECT
                month, day, year,
                max_air_temperature, min_air_temperature,
                precip,
                dew_point,
                sky_cover,
                wind_speed,
                wind_direction,
                solar_radiation,
                storm_type_id,
                potential_et,
                actual_et,
                input_units_code
            FROM climate_nldas2
            WHERE 
                station_id = 34617
            AND date >= '2000-01-01' AND date <= '2000-12-31';
            ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [14]:
from pyagnps.utils import get_bounds_from_cells

bounds = get_bounds_from_cells(watershed0.cells_geometry)

C:\Users\Luc\projects\pyagnps\src\pyagnps\utils.py:59: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bounds.geometry = bounds.geometry.buffer(buffer)


In [15]:
bounds.explore()